## Создание словаря используемых в ВК эмодзи на основе их тональной и смысловой окраски

In [1]:
!pip install --proxy 10.0.0.1:3128 nltk gensim pandas pymorphy2 emoji numpy

You should consider upgrading via the 'c:\users\admin\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


 --upgrade pip nltk gensim pandas pymorphy2 emoji numpy

In [2]:
!pip install pymorphy2

You should consider upgrading via the 'c:\users\admin\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


### Импорт необходимых библиотек

In [3]:
import pandas as pd
import nltk
from nltk.corpus import brown
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from gensim.models import Word2Vec as wv
from gensim.models import KeyedVectors
import pymorphy2
from pymorphy2 import MorphAnalyzer
import emoji
import zipfile
import numpy as np

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [7]:
nltk.download('averaged_perceptron_tagger_ru')

[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_ru is already up-to-
[nltk_data]       date!


True

In [8]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

## Подготовка датасета

### Чтение датасетов

#### df1

In [30]:
colnames = ['index', 'comment']

In [31]:
df1 = pd.read_csv('datasets/1.csv', names=colnames, header=None)

In [32]:
df1 = df1.dropna()

In [33]:
df1 = df1.drop(columns = ['index'],axis = 1)

In [34]:
df1

,comment
2,"[id211389315|Александр], норм"
3,😁
4,Воронка как цветок на воде 💮
5,Сша фашизм
6,Какие злодеи. Целых три месяца ждали.
...,...
5698,"[id619328383|Вера],"
5699,"[id252865345|Дмитрий], но именно все производс..."
5700,"[id252865345|Дмитрий],"
5701,"[id252865345|Дмитрий], эти картинки повесь в с..."


#### df2

In [35]:
df2 = pd.read_csv('datasets/2.csv', names=colnames, header=None)

In [36]:
df2 = df2.dropna()

In [37]:
df2 = df2.drop(columns = ['index'],axis = 1)

In [38]:
df2

,comment
1,Сменил снайпу на нож.\r\n-4750$
3,скинул дроп
4,А они её потом домой утащили???🙄🤔
5,Дропнул авик сильверам
7,"А у меня нет винтовки, но есть телефон всего з..."
...,...
2615,"[id677202852|Miles], ждём взлома"
2616,"[id677202852|Miles], они скорее всего это в оф..."
2617,Пойду Терминатора пересмотрю...
2618,"[id660356519|Егор], петух"


#### df3

In [39]:
df3 = pd.read_csv('datasets/3.csv', names=colnames, header=None)

In [40]:
df3 = df3.dropna()

In [41]:
df3 = df3.drop(columns = ['index'],axis = 1)

In [42]:
df3

,comment
1,Вот с#ка! Как жалко мальчишку! Чтоб папашу так...
2,"[id768749946|Ольга], Пугина ты опять не похмел..."
3,"[id768749946|Ольга], хахлоиды бандеровские...🐷"
4,"[id782850651|Наталья], гамасеки пиндоские"
5,"[id782850651|Наталья], вообще-то это Киргизия...."
...,...
5010,"[id242635060|Сергей], можно жить и не тужить н..."
5011,"[id435789502|Siji], никто и не глумится."
5012,"Не верится ему ,нам тоже не верится"
5013,"[id642424831|Дмитрий], а украинцы верят..всему😏"


#### df4

In [43]:
df4 = pd.read_csv('datasets/top_novostnoy.csv', names=colnames, header=None)

In [44]:
df4 = df4.dropna()

In [45]:
df4 = df4.drop(columns = ['index'],axis = 1)

In [46]:
df4

,comment
1,ДВУХГОЛОВЫЙ ОРЁЛ НЕ ИМЕЕТ ПОТОМСТВА!!!ПОЛЬШЕ П...
2,"[id761961298|Volodymyr], ты невнимательный там..."
3,МОЛОДЕЦ женщина! Когда наконец то русский наро...
4,"[id492418339|Юрий], вы думаете, это обращение ..."
5,Админ группы-конченный ты
...,...
2768,Шняги на лбу нехвотает🤣🤣🤣
2769,Просто хвостик нужно тугой сделать и все ок! 👌
2770,"[id570536144|Татьяна], 🤣🤣🤣"
2771,"[id570536144|Татьяна], ага собачий 😂😂😂😂"


### Слияние датафреймов в один

In [47]:
comments = pd.concat([df1, df2, df3, df4], axis=0)

In [48]:
comments = comments.reset_index(drop=True)

In [49]:
comments

,comment
0,"[id211389315|Александр], норм"
1,😁
2,Воронка как цветок на воде 💮
3,Сша фашизм
4,Какие злодеи. Целых три месяца ждали.
...,...
15429,Шняги на лбу нехвотает🤣🤣🤣
15430,Просто хвостик нужно тугой сделать и все ок! 👌
15431,"[id570536144|Татьяна], 🤣🤣🤣"
15432,"[id570536144|Татьяна], ага собачий 😂😂😂😂"


##### Предобработка комментариев

Удаление из комментраиев-ответов имени и идентификатора пользователя [id255548341|Анастасия], которому адресован ответ 

In [50]:
pattern = r'\[(.*?)\]'

def reply_delete(doc):
    doc = re.sub(pattern, ' ', str(doc))
    return doc

In [51]:
comments['comment'] = comments['comment'].apply(reply_delete)

In [52]:
comments

,comment
0,", норм"
1,😁
2,Воронка как цветок на воде 💮
3,Сша фашизм
4,Какие злодеи. Целых три месяца ждали.
...,...
15429,Шняги на лбу нехвотает🤣🤣🤣
15430,Просто хвостик нужно тугой сделать и все ок! 👌
15431,", 🤣🤣🤣"
15432,", ага собачий 😂😂😂😂"


In [53]:
def divide_emoji(doc):
    for c in str(doc):
      if emoji.is_emoji(c):
          i = doc.index(c)
          doc = ' ' + doc[:i] + ' ' + doc[i:]
    return doc

In [54]:
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()

def lemmatize(doc):
    tokens = []
    doc = re.sub(patterns, ' ', str(doc))
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            tokens.append(token)
    return tokens

In [55]:
comments['comment'] = comments['comment'].apply(divide_emoji)

In [62]:
comments['comment'] = comments['comment'].apply(lemmatize)

In [57]:
comments = comments.dropna()

In [68]:
comments

,comment
0,[норма]
1,[😁]
2,"[воронка, цветок, вода, 💮]"
3,"[сша, фашизм]"
4,"[злодей, целый, месяц, ждать]"
...,...
15429,"[шняга, лоб, нехвотаять, 🤣🤣🤣]"
15430,"[просто, хвостик, нужно, тугой, сделать, около..."
15431,[🤣🤣🤣]
15432,"[ага, собачий, 😂😂😂😂]"


## Слияние с датасетом по Алтайскому краю

In [59]:
colnames = ['index', 'comment']

In [60]:
altai = pd.read_csv('datasets/altai_comments.csv', names=colnames, header=0, sep='\t')

In [63]:
altai = altai.drop(columns = ['index'],axis = 1)

In [65]:
altai

,comment
0,"['ледокол', '🤣']"
1,"['красавец', 'какой', 'удивительный', 'природа..."
2,"['какой', 'это', 'человек', 'маленько', 'дотяг..."
3,"['шикарный', '👍']"
4,['краснокнижный']
...,...
5141,"['с', 'фамилия', '🙄']"
5142,"['наш', 'день', 'повезти', 'мужик', 'фамилия',..."
5143,"['чудо', 'чудной', 'диво', 'дивный']"
5144,"['пригреться', 'уснуть', 'свариться', 'лягушка..."


In [66]:
dataset_vk = pd.concat([comments, altai], axis=0)

In [69]:
dataset_vk = dataset_vk.reset_index(drop=True)

In [70]:
dataset_vk

,comment
0,[норма]
1,[😁]
2,"[воронка, цветок, вода, 💮]"
3,"[сша, фашизм]"
4,"[злодей, целый, месяц, ждать]"
...,...
20575,"['с', 'фамилия', '🙄']"
20576,"['наш', 'день', 'повезти', 'мужик', 'фамилия',..."
20577,"['чудо', 'чудной', 'диво', 'дивный']"
20578,"['пригреться', 'уснуть', 'свариться', 'лягушка..."


In [71]:
dataset_vk.to_csv('datasets/dataset_vk.csv', sep='\t')

# Статистика

In [43]:
total = comments.count()[0]

In [44]:
def split(s):
    return [char for char in s]


def check(comment):
    flag=0
    for item in comment:
        for c in split(item):
            if emoji.is_emoji(c):
                flag+=1
    if flag > 0:
        return(comment)
    return None 

In [45]:
comments['comment'] = comments['comment'].apply(check)

In [46]:
comments = comments.dropna()

In [47]:
comments

,comment
0,"[ледокол, 🤣]"
1,"[красавец, какой, удивительный, природа, 😊, ст..."
3,"[шикарный, 👍]"
8,"[каков, красавец, 😍]"
9,"[зачем, ❓, собраться, убивать, это, тот, стрел..."
...,...
5079,"[репост, приветствоваться, 😉]"
5090,[🤣🤣🤣🤣🤣🤣🤣]
5099,[👍👍👍👍👍]
5114,"[платить, сгк, новый, ахта, 🤣, старый, арестов..."


In [48]:
comments.count()[0]/total

0.19257675864749318

In [49]:
def collect_emojis(comment):
    emojis=list()
    for item in comment:
        for c in split(item):
            if emoji.is_emoji(c):
                emojis.append(c)
    return emojis 

In [50]:
emojis = comments['comment'].apply(collect_emojis).to_list()

In [51]:
smiles = list()

for emoji in emojis:
    for e in emoji:
        smiles.append(e)

In [52]:
smiles

['🤣',
 '😊',
 '👍',
 '😍',
 '❓',
 '🤣',
 '😂',
 '👍',
 '😉',
 '🤣',
 '🤦',
 '♂',
 '🙏',
 '😁',
 '🧹',
 '👍',
 '🏻',
 '😆',
 '🤣',
 '🤣',
 '🤣',
 '💩',
 '🤣',
 '🤣',
 '🤣',
 '😄',
 '😀',
 '⚠',
 '🤦',
 '♂',
 '🤷',
 '🤔',
 '🤣',
 '😁',
 '🤣',
 '😒',
 '🤣',
 '🤦',
 '♂',
 '✌',
 '☝',
 '🤗',
 '😅',
 '👌',
 '🏻',
 '🤔',
 '🤣',
 '🤗',
 '☀',
 '😐',
 '😉',
 '😈',
 '🤷',
 '🤷',
 '♂',
 '📲',
 '📲',
 '🤣',
 '🤷',
 '🤣',
 '🤦',
 '♂',
 '😬',
 '😬',
 '😬',
 '🥺',
 '🤔',
 '😭',
 '🤔',
 '💯',
 '🤔',
 '🙏',
 '😢',
 '🙏',
 '🤐',
 '🤷',
 '♀',
 '🤷',
 '♀',
 '🤷',
 '♀',
 '😭',
 '🙏',
 '😭',
 '😭',
 '😭',
 '😥',
 '😉',
 '🔥',
 '🤔',
 '🤪',
 '🤣',
 '👍',
 '💩',
 '😄',
 '🤣',
 '🤣',
 '🤣',
 '🤷',
 '♂',
 '😡',
 '✌',
 '🤔',
 '😳',
 '😲',
 '🙄',
 '🤧',
 '🤷',
 '♂',
 '🤷',
 '♀',
 '😅',
 '👍',
 '🤣',
 '🤣',
 '🤣',
 '🤣',
 '🤣',
 '😉',
 '👍',
 '✅',
 '✅',
 '✅',
 '😉',
 '🤷',
 '😞',
 '🔥',
 '😅',
 '🤦',
 '♂',
 '🤔',
 '🤣',
 '🤣',
 '😎',
 '🤔',
 '🤧',
 '🙄',
 '😳',
 '😲',
 '🌱',
 '🌱',
 '🌱',
 '🌱',
 '🌱',
 '🌱',
 '🌱',
 '🌱',
 '🌱',
 '😂',
 '🤷',
 '😞',
 '😞',
 '🤷',
 '🤷',
 '😂',
 '👏',
 '😒',
 '❓',
 '🤗',
 '✌',
 '✨',
 '😅',
 '😄',
 '🌴',
 '🌞',
 '🤣',
 '😔'

In [53]:
len(smiles)

1912

In [54]:
from collections import Counter

In [55]:
cnt = Counter(smiles)

In [56]:
len(cnt)

147

In [57]:
uniq = cnt.most_common(120)

Процент уникальных эмодзи среди 5141 сообщений (из которых 1912 смайликов, в 991 сообщении)

In [58]:
len(cnt)/len(smiles)

0.07688284518828452

In [59]:
type(uniq)

list

## Подготовка словаря с наиболее часто встречающимися эмодзи и их контекстом по их тональной окраске

Это делается для дальнейшего обучения модели Word2vec

In [60]:
emoji_dict = pd.DataFrame(columns = ['Emoji', 'Description'])

In [61]:
uniq

[('🤣', 444),
 ('👍', 158),
 ('😂', 134),
 ('♂', 105),
 ('🤦', 85),
 ('😁', 80),
 ('🤷', 61),
 ('🤔', 56),
 ('👏', 54),
 ('😢', 41),
 ('😄', 33),
 ('♀', 32),
 ('🙏', 27),
 ('😅', 26),
 ('😉', 23),
 ('😆', 21),
 ('☝', 20),
 ('💪', 17),
 ('🏻', 16),
 ('😭', 16),
 ('😊', 15),
 ('😡', 15),
 ('☕', 15),
 ('🤗', 14),
 ('😳', 14),
 ('😏', 14),
 ('👌', 12),
 ('🔥', 11),
 ('🙈', 11),
 ('😰', 11),
 ('😒', 10),
 ('🤧', 10),
 ('🤡', 10),
 ('✌', 9),
 ('🙄', 9),
 ('😎', 9),
 ('🌱', 9),
 ('🤪', 7),
 ('🤭', 7),
 ('🙃', 7),
 ('☺', 7),
 ('✊', 7),
 ('🏼', 7),
 ('😃', 7),
 ('😬', 6),
 ('✨', 6),
 ('🙂', 6),
 ('🤬', 6),
 ('😨', 6),
 ('😀', 5),
 ('😞', 5),
 ('😔', 5),
 ('🧐', 5),
 ('💩', 4),
 ('🤝', 4),
 ('👋', 4),
 ('🦊', 4),
 ('😤', 4),
 ('👿', 4),
 ('😍', 3),
 ('😐', 3),
 ('💯', 3),
 ('😲', 3),
 ('✅', 3),
 ('😜', 3),
 ('😩', 3),
 ('😵', 3),
 ('❤', 3),
 ('🤙', 3),
 ('💋', 3),
 ('👅', 3),
 ('😺', 3),
 ('😇', 3),
 ('😱', 3),
 ('👎', 3),
 ('🌚', 3),
 ('❓', 2),
 ('🧹', 2),
 ('📲', 2),
 ('🤐', 2),
 ('🌞', 2),
 ('🤨', 2),
 ('🏕', 2),
 ('😕', 2),
 ('😠', 2),
 ('‼', 2),
 ('💚', 2),
 ('🏾',

In [62]:
for i in uniq:
    emoji_dict.loc[ len(emoji_dict.index )] = [i[0], str(i[1])]

In [63]:
emoji_dict

,Emoji,Description
0,🤣,444
1,👍,158
2,😂,134
3,♂,105
4,🤦,85
...,...,...
115,😴,1
116,🫴,1
117,🌇,1
118,🏢,1


## Изменение каждой записи

### Для формирования ассоциаций использовал ChatGPT

In [64]:
emoji_dict.loc[0]

Emoji            🤣
Description    444
Name: 0, dtype: object

In [65]:
emoji_dict.loc[0]['Description'] = ['смешно', 'усморительно', 'ржачно', 'угарно', 'умора']

In [66]:
emoji_dict

,Emoji,Description
0,🤣,"[смешно, усморительно, ржачно, угарно, умора]"
1,👍,158
2,😂,134
3,♂,105
4,🤦,85
...,...,...
115,😴,1
116,🫴,1
117,🌇,1
118,🏢,1


## Сохранение в csv

In [64]:
emoji_dict.to_csv('emoji_dict.csv', sep=',')